# Challenge 2: Exploratory Data Analysis (EDA)

In [66]:
# Data manipulation
import pandas as pd # data manipulation and dataframes
import numpy as np # arrays manipulation and mathematical operations

# Fuzzy string matching
from fuzzywuzzy import fuzz

In [67]:
# Pandas configuration
pd.set_option('display.max_columns', None)  # shows all columns
pd.set_option('display.max_colwidth', None)  # shows all cell content

## Some cleaning

In [68]:
# Reads csvs
initial_csv = pd.read_csv('../data/initial-dia.csv', index_col = 0)
scrapped_csv = pd.read_csv('../data/scraped-dia.csv', index_col = 0)

# Index
initial_csv.reset_index(drop = False, inplace = True) # resets index named by url

# Nulls
initial_csv.isnull().sum() # 3053473 description, 5480 reference_unit

# Drops
initial_csv.drop(columns=['description'], axis = 1, inplace = True) # descrition column (all values are null)
initial_csv.drop(columns=['product_id'], axis = 1, inplace = True) # product_id column (not used information)
scrapped_csv.drop(columns=['subcategory'], axis = 1, inplace = True) # subcategory column (not used information)

# Gets YYYY-MM-DD format
initial_csv['insert_date'] = initial_csv['insert_date'].str.split(' ', expand = True).get(0) 

# Matches reference_unit column content between both dfs
units = scrapped_csv['reference_unit'].tolist()

new_units = []

for u in units:

    if u == 'kilo':
        new_units.append('kg')

    elif u == 'unidad':
        new_units.append('ud')

    elif u == 'litro':
        new_units.append('l')

    elif u == 'metro':
        new_units.append('m')

    elif u == 'lavado':
        new_units.append('lavado')

    else:
        new_units.append(np.nan)

scrapped_csv['reference_unit'] = new_units

# Adjusts column dtypes
initial_csv['insert_date'] = pd.to_datetime(initial_csv['insert_date']) # to datetime
scrapped_csv['insert_date'] = pd.to_datetime(scrapped_csv['insert_date']) # to datetime
scrapped_csv.iloc[2698, scrapped_csv.columns.get_loc('reference_price')] = '13.82' # corrects supermarket labelling error
scrapped_csv['reference_price'] = pd.to_numeric(scrapped_csv['reference_price']) # to float

# No url info for scraped csv
scrapped_csv['url'] = np.nan # wip

# Same column order
scrapped_csv = scrapped_csv.reindex(columns = initial_csv.columns)

## Matching csvs by category with FuzzyWuzzy

In [ ]:
# paso a listas las cols de categoría del dataframe scrapeado y del inicial
cat_nuevas = scrapped_csv['category'].unique().tolist()
cat_antiguas = initial_csv['category'].unique().tolist()

def cat_max(col, limpita):
    '''
    Devuelve categoría tras aplicar fuzzywuzzy
    '''

    maximo = 0

    for l in limpita:

        parecido = fuzz.ratio(col, l)

        if parecido > maximo:

            maximo = parecido
            
            categoria = l

    return categoria

# splitea los valores de las categorías scrapeadas por '_'
spliteada = []

for s in cat_nuevas:

    a = s.split('_')

    spliteada.append(a)

# flatea la lista para pasar de lista de listas a lista de strs
flat_list = []

for sublist in spliteada:

    for item in sublist:

        flat_list.append(item)

# apendea en una lista todas las palabras eliminando aquellas de poco interés para el fuzzywuzzy
limpita = []

for f in flat_list:

    # se evitan los nexos entre palabras
    if f == 'y' or f == 'e' or f == 'con':
        pass

    else:
        limpita.append(f)


# añade a lo lista limpia anterior algunas categorías especiales como navidad y otros términos complicados de clasificar
apendeos = ['gluten', 'lacteos', 'navidad', 'dieteticos', 'solidario', 'aceitunas', 'sal', 
            'cuidado', 'internacional', 'mermeladas', 'licores', 'sopas', 'espumosos']

for a in apendeos:
    
    limpita.append(a)

# splitea los valores de las categorías iniciales por '_'
aux =[]

for c in cat_antiguas:

    aux.append(c.split('_'))


# descarta palabras que dificultan el matcheo por fuzzywuzzy
aux2 = []
word = ''

descartes = ['y', 'e', 'vinagre', 'con', 'del', 'al', 'de', 'dia', 'alimentacion', 'fresco', 'frescos', 
            'desayuno', 'despensa', 'bodega', 'drogueria', 'desayunos', 'dulces',
            'cocina', 'personal', 'soy', 'bano', 'corporal', 'preparacion', 'bebidas']

for a in aux:

    for i in a:
        
        if i in descartes:
            pass

        else:
            word += i + ' '
    
    aux2.append(word.rstrip(' '))

    word = ''

# para cada una de las listas de strs de categorías iniciales tras los descartes se selecciona solo la primera palabra, la más representativa
aux3 = []

for a in aux2:

    b = a.split(' ')
    aux3.append(b[0])

# genera un df con la palabra representativa de las categorías iniciales y con las resultantes del fuzzywuzzy
df = pd.DataFrame()
df['antiguas'] = pd.DataFrame(aux3)
df['fuzzcat'] = df.apply(lambda x: cat_max(x['antiguas'], limpita), axis = 1)

# YA SE HAN RELACIONADO LAS CATEGORÍAS CON FUZZYWUZZY
# AHORA HAY QUE HACER EL PROCESO CONTRARIO DE RELACIONAR LAS PALABRAS CLAVES MATCHEADAS CON LOS NOMBRES DE LAS CATEGORÍAS

# se repite el proceso de descartar nexos entre palabras para las categorías scrapeadas, pero se mantiene la estructura de lista de listas y no se flatea como anteriormente
definitiva = []
aux = []
for sublist in spliteada:

    for f in sublist:

        if f == 'y' or f == 'e' or f == 'con':
            pass

        else:
            aux.append(f)
    definitiva.append(aux)
    aux =[]


def convertir_lista_a_diccionario(lista, claves):
    '''
    Convierte una lista de listas en un diccionario según unas keys dadas
    '''
    dictio = {}
    
    for i, sublist in enumerate(lista):

        clave = claves[i]
        dictio[clave] = sublist
    
    return dictio

# crea un diccionario {categorias scrapeadas}: [palabras clave de la categoría usadas para fuzzywuzzy]
# ejemplo {azucar_chocolates_y_caramelos}: [azucar, chocolates, caramelos]
lista_de_listas = definitiva
dictio = convertir_lista_a_diccionario(lista_de_listas, cat_nuevas)

# relaciona los apendeos con categorías existentes o crea nuevas categorías
# nuevas categorías:
dictio['gluten'] = ['gluten'] # sorry celiacos
dictio['solidario'] = ['solidario']
dictio['navidad'] = ['navidad']
dictio['internacional'] = ['internacional']

# relaciona términos con categorías existentes:
dictio['leche_huevos_y_mantequilla'].append('lacteos')
dictio['galletas_bollos_y_cereales'].append('dieteticos')
dictio['patatas_fritas_encurtidos_y_frutos_secos'].append('aceitunas')
dictio['aceites_salsas_y_especias'].append('sal')
dictio['perfumeria_higiene_salud'].append('cuidado')
dictio['azucar_chocolates_y_caramelos'].append('mermeladas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('licores')
dictio['conservas_caldos_y_cremas'].append('sopas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('espumosos')

# transforma las palabras clave de fuzzywuzzy en las categorías scrapeadas categorías scrapeadas
# ejemplo leche: leche_huevos_y_mantequilla
listiña = []

for i in df['fuzzcat']:

    for key, values in dictio.items():

        if i in values:

            listiña.append(key)

# añade al df
df['feas'] = cat_antiguas
df['transformadas'] = listiña

transformadas = df['transformadas'].tolist()
feas = df['feas'].unique().tolist()

# diccionario para transformas las categorías iniciales en las categorías scrapeadas
# {categoría inicial}: categoría scrapeada
hada = {}

for i in range(0, len(transformadas)):

    hada[feas[i]] = transformadas[i]

# map
initial_csv['nuevas_categorias'] = initial_csv['category'].map(hada)

# drop old and rename
initial_csv.drop(columns=['category'], axis = 1, inplace = True)
initial_csv.rename(columns={"nuevas_categorias": "category"}, inplace=True)